In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

**download dataset and add label to for close column**



In [14]:
xmr = yf.Ticker("XMR-USD")
df_xmr = yf.download(tickers = "XMR-USD",
                     period = "max",
                     interval = "1d").reset_index()

[*********************100%%**********************]  1 of 1 completed


In [15]:
def add_label(df, column, label):
  df[label] = np.where(df[column] < df[column].shift(-1), 1, 0)
  return df

In [16]:
df_xmr = add_label(df_xmr, 'Close', 'Close_Increase')

In [9]:
df_xmr

,Date,Open,High,Low,Close,Adj Close,Volume,Close_Increase
0,2017-11-09,112.531998,123.404999,112.219002,120.779999,120.779999,86864600,0
1,2017-11-10,121.344002,121.665001,101.757004,105.585999,105.585999,84614000,1
2,2017-11-11,105.750000,127.106003,103.877998,119.615997,119.615997,107708000,1
3,2017-11-12,119.597000,133.675003,110.617996,123.856003,123.856003,144948000,0
4,2017-11-13,128.960007,136.528000,120.921997,123.402000,123.402000,116200000,0
...,...,...,...,...,...,...,...,...
2158,2023-10-07,151.988235,155.247528,151.100983,155.212143,155.212143,61159796,1
2159,2023-10-08,155.219528,156.482040,153.763336,156.191223,156.191223,59858985,0
2160,2023-10-09,156.190552,156.192932,151.908813,153.795502,153.795502,61203300,0
2161,2023-10-10,153.790405,154.468155,152.097107,152.215652,152.215652,54504811,1


In [33]:
df_xmr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2163 entries, 0 to 2162
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            2163 non-null   datetime64[ns]
 1   Open            2163 non-null   float64       
 2   High            2163 non-null   float64       
 3   Low             2163 non-null   float64       
 4   Close           2163 non-null   float64       
 5   Adj Close       2163 non-null   float64       
 6   Volume          2163 non-null   int64         
 7   Close_increase  2163 non-null   int64         
dtypes: datetime64[ns](1), float64(5), int64(2)
memory usage: 135.3 KB


In [27]:
df_xmr = df_xmr.drop('Date', axis=1)

In [29]:
X = df_xmr.loc[:, df_xmr.columns != 'Close_Increase']
y = df_xmr['Close_Increase']

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = RandomForestRegressor(n_estimators=1000, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [33]:
print(mean_squared_error(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

0.28334560092449923
0.49372419106317417
-0.13360040325797895


In [34]:
X_norm = (X - X.min()) / (X.max() - X.min())

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, random_state=42)

reg = RandomForestRegressor(n_estimators=1000, random_state=42)
reg.fit(X_train, y_train)

y_pred = reg.predict(X_test)


In [36]:
print(mean_squared_error(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

0.2831310647149461
0.49348073959938366
-0.13274209336056253


In [37]:
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [39]:
print(mean_squared_error(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))
print(r2_score(y_test, y_pred))

0.4930662557781202
0.4930662557781202
-0.9726443768996962
